In [1]:
from Hamil_search import *
from qiskit.quantum_info import SparsePauliOp, Statevector
from scipy.sparse.linalg import eigsh, LinearOperator
from scipy import linalg as LA

In [ ]:
n = 8
config = {"target": 0, "distance": 2, "depth": 1, "thres": 0}
p = {0: "X", 1: "Z", 2: "Y"}
m = n // 2
res = []
for i in range(m):
    dex = p[i % 2]
    if m % 2 == 1 and i == m - 1:
        dex = "Y"
    res.append([f"{dex}{i*2}*{dex}{i*2+1}", f"-1*{dex}{(i*2+2)%n}*{dex}{(i*2+3)%n}"])
print(res)

Ml = [sum([pauliExpr2Mat(n, x) for x in xl]) for xl in res]
eff = [2, 3, 4, 8]
Hm = sum([eff[i] * Ml[i] for i in range(m)])
space = getSpace(n, Hm, config)

print(space.shape[1])

[['X0*X1', '-1*X2*X3'], ['Z2*Z3', '-1*Z4*Z5'], ['X4*X5', '-1*X6*X7'], ['Z6*Z7', '-1*Z0*Z1']]
16


In [2]:
n = 3
H = pauliStr2mat(n, "X2")
Ht = SparsePauliOp(["IIX"], np.array([1]))
# print(H)

v = np.array(list(range(2**n)))
print(v)
print(H @ v)
vp = v.reshape((2, 2, 2))
state = Statevector(v)
print(state.data)
state = state.evolve(Ht)
print(state.data)

SparsePauliOp(['IIX'],
              coeffs=[1.+0.j])
[0 1 2 3 4 5 6 7]
[1. 0. 3. 2. 5. 4. 7. 6.]
[0.+0.j 1.+0.j 2.+0.j 3.+0.j 4.+0.j 5.+0.j 6.+0.j 7.+0.j]
[1.+0.j 0.+0.j 3.+0.j 2.+0.j 5.+0.j 4.+0.j 7.+0.j 6.+0.j]


In [9]:
n = 9


def getXMapper(index):
    result = []
    for i in range(2**n):
        result.append(i ^ 2 ** (n - index - 1))
    return result


Xmapper = [getXMapper(i) for i in range(n)]


def applyX(v, index):
    return v[Xmapper[index]]


def applyZ(vInput, index):
    v = np.copy(vInput)
    shift = n - index - 1
    for i in range(2**n):
        if (i >> shift) % 2 == 1:
            v[i] = -v[i]
    return v


def pauli2QiskitStr(terms, num_qubit):
    result = ["I" for _ in range(num_qubit)]
    for t in terms:
        result[t[1]] = t[0]
    return "".join(result)


def Hstr2QiskitPauliOp(Hstr, num_qubit):
    result = []
    effs = []
    for term in Hstr:
        eff, paulis = splitPaulis(term)
        effs.append(eff)
        result.append(pauli2QiskitStr(paulis, num_qubit))
    return SparsePauliOp(result, np.array(effs))


def applyH(H, num_qubit, v):
    state = Statevector(v)
    state = state.evolve(H)
    return state.data


def applyHo(Hstr, v, inverse=False):
    result = np.zeros(2**n)
    for term in Hstr:
        vcopy = np.copy(v)
        eff, paulis = splitPaulis(term)
        if inverse:
            paulis = paulis[-1::-1]
        for p in paulis:
            if p[0] == "X":
                vcopy = applyX(vcopy, p[1])
            if p[0] == "Z":
                vcopy = applyZ(vcopy, p[1])
        result += eff * vcopy
    return result

In [ ]:
Mtype = np.float64
v = np.array(list(range(2**n)))
H = [
    "Z0*Z3",
    "X6*X7",
    "X1*X2",
    "Z5*Z8",
    "-4*I0",
    "1*X3*X4",
    "-1*X0*X1",
    "1*Z4*Z5",
    "-1*Z1*Z2",
    "1*Z6*Z7",
    "-1*Z3*Z4",
    "1*X7*X8",
    "-1*X4*X5",
]

H1 = [
    "Z0*Z1",
    "Z2*Z3",
    "X4*X8",
    "X7*X11",
    "Z12*Z13",
    "Z14*Z15",
    "-6*I0",
    "X0*X1",
    "-1*X4*X5",
    "Z1*Z2",
    "-1*Z5*Z6",
    "X2*X3",
    "-1*X6*X7",
    "Z4*Z5",
    "-1*Z8*Z9",
    "X5*X6",
    "-1*X9*X10",
    "Z6*Z7",
    "-1*Z10*Z11",
    "X8*X9",
    "-1*X12*X13",
    "Z9*Z10",
    "-1*Z13*Z14",
    "X10*X11",
    "-1*X14*X15",
]

# HP = Hstr2QiskitPauliOp(H, n)
# HM = sum([pauliExpr2Mat(n, s) for s in H])
# vp = applyH(HP, n, applyH(HP.transpose(), n, v))
# vm = HM @ HM.T @ v
# print(vp - vm)

In [11]:
# Hlar = LinearOperator(
#     shape=(2**n, 2**n),
#     matvec=lambda v: applyH(HP, n, applyH(HP.transpose(), n, v)),
#     dtype=Mtype,
# )

Hlar = LinearOperator(
    shape=(2**n, 2**n),
    matvec=lambda v: applyHo(H, applyHo(H, v, inverse=True)),
    dtype=Mtype,
)
w, ev = eigsh(Hlar, 3, which="LM")
w.sort()
print(w)
print(n)
shift = w[-1]

[146.35760392 168.68602475 168.68602475]
9


In [ ]:
kets = [
    "000000000",
    "000000110",
    "011000000",
    "011000110",
    "110110000",
    "110110110",
    "101110000",
    "101110110",
    "000011011",
    "000011101",
    "011011011",
    "011011101",
    "110101011",
    "110101101",
    "101101011",
    "101101101",
]
v0 = ket2Vec(n, kets) / (len(kets)) ** 0.5

In [ ]:
# H = ["-1*Z0*X1", "Z1*X2"]

print(f"shift is {shift}")
# HM = sum([pauliExpr2Mat(n, s) for s in H])
# I = np.identity(HM.shape[0])
# Hs = HM @ HM.T - shift * I
# HL = LinearOperator(shape=HM.shape, matvec=lambda v: Hs @ v, dtype=HM.dtype)
HLT = LinearOperator(
    shape=(2**n, 2**n),
    matvec=lambda v: applyHo(H, applyHo(H, v, inverse=True)) - shift * v,
    dtype=Mtype,
)


w, ev = eigsh(HLT, 3, which="LM", maxiter=10000, v0=v0)


print(w + shift)

shift is 168.6860247513159
[-9.94759830e-13  2.27373675e-13  9.57162442e-03]


In [38]:
7 >> 2

1